# Calculate Percentage Distance

Calculating percentage distance from company metric to industry average

In [7]:
from ib_insync import *
import pandas as pd
import time
import nasdaqdatalink
import sys
from datetime import datetime, timedelta
import numpy as np

sys.path.append('/Users/markwindsor/Desktop/atlas_trade')
csv_universe_path = '/Users/markwindsor/Desktop/atlas_trade/src/data/universe.csv'
csv_industry_avg_path = '/Users/markwindsor/Desktop/atlas_trade/src/data/industry_metrics.csv'

nasdaqdatalink.ApiConfig.api_key = 'zxLNKQydu_qNXQ2tZ7vz'

In [8]:
universe_df = pd.read_csv(csv_universe_path)
industry_avg_df = pd.read_csv(csv_industry_avg_path).set_index('industry')

universe_df 

,None,table,permaticker,ticker,name,exchange,isdelisted,category,cusips,siccode,...,currency,location,lastupdated,firstadded,firstpricedate,lastpricedate,firstquarter,lastquarter,secfilings,companysite
0,0,SF1,640388,FMST,FOREMOST LITHIUM RESOURCE & TECHNOLOGY LTD,NASDAQ,N,Canadian Common Stock Primary Class,345510101 345510200,1090,...,CAD,British Columbia; Canada,2023-08-23,2023-08-23,2023-08-22,2023-08-29,2021-03-31,2023-03-31,https://www.sec.gov/cgi-bin/browse-edgar?actio...,NaN
1,1,SF1,640365,NNAGU,99 ACQUISITION GROUP INC,NASDAQ,N,Domestic Common Stock,NaN,6770,...,USD,Maryland; U.S.A,2023-08-21,2023-08-21,2023-08-18,2023-08-29,2022-12-31,2023-03-31,https://www.sec.gov/cgi-bin/browse-edgar?actio...,NaN
2,2,SF1,640361,SEZL,SEZZLE INC,NASDAQ,N,Domestic Common Stock,U8180P106,7389,...,USD,Minnesota; U.S.A,2023-08-17,2023-08-17,2023-08-17,2023-08-28,2019-12-31,2023-06-30,https://www.sec.gov/cgi-bin/browse-edgar?actio...,http://www.sezzle.com
3,3,SF1,640355,SRM,SRM ENTERTAINMENT INC,NASDAQ,N,Domestic Common Stock,NaN,3944,...,USD,Florida; U.S.A,2023-08-16,2023-08-16,2023-08-15,2023-08-29,2021-12-31,2023-03-31,https://www.sec.gov/cgi-bin/browse-edgar?actio...,https://www.srmentertainment.com
4,4,SF1,640351,SPGC,SACKS PARENTE GOLF INC,NASDAQ,N,Domestic Common Stock,NaN,3949,...,USD,California; U.S.A,2023-08-15,2023-08-15,2023-08-15,2023-08-29,2020-12-31,2022-12-31,https://www.sec.gov/cgi-bin/browse-edgar?actio...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4928,9981,SF1,123695,TPHS,TRINITY PLACE HOLDINGS INC,NYSEMKT,N,Domestic Common Stock,89656D101,6512,...,USD,New York; U.S.A,2023-08-15,2017-07-19,2015-12-21,2023-08-29,2015-03-31,2023-06-30,https://www.sec.gov/cgi-bin/browse-edgar?actio...,http://www.tphs.com
4929,9988,SF1,123688,SPI,SPI ENERGY CO LTD,NASDAQ,N,ADR Common Stock,G8651P110 78470H109 G8651P201 G8651P102 83490A100,3674,...,USD,China,2023-08-18,2017-07-20,2016-01-19,2023-08-29,2014-12-31,2023-06-30,https://www.sec.gov/cgi-bin/browse-edgar?actio...,http://www.spigroups.com
4930,9993,SF1,123683,CLLS,CELLECTIS SA,NASDAQ,N,ADR Common Stock,15117K103,2836,...,USD,France,2023-08-07,2017-07-20,2015-03-25,2023-08-29,2013-12-31,2023-06-30,https://www.sec.gov/cgi-bin/browse-edgar?actio...,http://www.cellectis.com
4931,9994,SF1,123682,IMRN,IMMURON LTD,NASDAQ,N,ADR Common Stock Primary Class,45254U101,2834,...,AUD,Australia,2023-03-01,2017-07-20,2017-06-09,2023-08-29,2015-06-30,2022-12-31,https://www.sec.gov/cgi-bin/browse-edgar?actio...,http://www.immuron.com.au


In [10]:
# Get dataframe of famaindustry and symbol
ticker_industry_df = universe_df[['ticker', 'famaindustry']]

# This is the resulting dataframe of this snippet. Will indclude all companies with their respective
# percentage distance from their metric and their industry average
percentage_dist = []

for index, row in ticker_industry_df.head(100).iterrows():
    # Grab industry average row by industry
    ticker = row['ticker']
    industry = row['famaindustry']

    if not isinstance(industry, str):
        continue

    row_values = {}

    row_values['ticker'] = ticker    
    row_values['industry'] = industry

    

    industry_average_row = industry_avg_df.loc[industry]
    
    if (industry_average_row.empty):
        continue

    # ==========================================================
    # Daily Table (SHARADAR/DAILY)
    # ==========================================================
    yesterday = datetime.now() - timedelta(days=1)
    yesterday_date = yesterday.strftime('%Y-%m-%d')

    daily = nasdaqdatalink.get_table('SHARADAR/DAILY', date=yesterday_date, qopts={"columns":['ticker', 'ev', 'evebit', 'evebitda', 'marketcap', 'pb', 'pe', 'ps']}, ticker=ticker)
    daily_df = pd.DataFrame(daily)

    if not daily_df.empty:
        row_values['evebit'] = ((daily_df['evebit'] - industry_average_row['evebit']) / industry_average_row['evebit']).values[0]
        row_values['evebitda'] = ((daily_df['evebitda'] - industry_average_row['evebitda']) / industry_average_row['evebitda']).values[0]
        row_values['pb'] = ((daily_df['pb'] - industry_average_row['pb']) / industry_average_row['pb']).values[0]
        row_values['pe'] = ((daily_df['pe'] - industry_average_row['pe']) / industry_average_row['pe']).values[0]
        row_values['ps'] = ((daily_df['ps'] - industry_average_row['ps']) / industry_average_row['ps']).values[0]
    else:
        print("Daily Data not available for calculate for: ", ticker)

    # ==========================================================
    # Metrics Table (SHARADAR/METRICS)
    # ==========================================================
    metrics = nasdaqdatalink.get_table('SHARADAR/METRICS',  qopts={"columns":['ticker', 'beta1y', 'beta5y', 'return1y', 'return5y', 'volume', 'volumeavg1m', 'volumeavg3m']}, ticker=ticker)
    metrics_df = pd.DataFrame(metrics)

    if not metrics_df.empty:
        row_values['beta1y'] = ((metrics_df['beta1y'] - industry_average_row['beta1y']) / industry_average_row['beta1y']).values[0]
        row_values['beta5y'] = ((metrics_df['beta5y'] - industry_average_row['beta5y']) / industry_average_row['beta5y']).values[0]
        row_values['return1y'] = ((metrics_df['return1y'] - industry_average_row['return1y']) / industry_average_row['return1y']).values[0]
        row_values['return5y'] = ((metrics_df['return5y'] - industry_average_row['return5y']) / industry_average_row['return5y']).values[0]
        row_values['volume'] = ((metrics_df['volume'] - industry_average_row['volume']) / industry_average_row['volume']).values[0]
        row_values['volumeavg1m'] = ((metrics_df['volumeavg1m'] - industry_average_row['volumeavg1m']) / industry_average_row['volumeavg1m']).values[0]
        row_values['volumeavg3m'] = ((metrics_df['volumeavg3m'] - industry_average_row['volumeavg3m']) / industry_average_row['volumeavg3m']).values[0]
    else:
        print("Metric Data not available for: ", ticker)

    # ==========================================================
    # Fundamentals Table (SHARADAR/SF!)
    # ==========================================================

    full_figure_list = ['revenue', 'netinc', 'bvps', 'rnd', 'fcf', 'equity', 'assetturnover', 'currentratio', 'de', 'divyield', 'eps', 'fcfps', 'grossmargin', 'ebitdamargin', 'sps', 'ros', 'roic', 'roe', 'roa', 'payoutratio']
    latest_observation_list = ['bvps', 'assetturnover', 'currentratio', 'de', 'divyield', 'eps', 'fcfps', 'grossmargin', 'ebitdamargin', 'sps', 'ros', 'roic', 'roe', 'roa', 'payoutratio']
    growth_rates_list = ['revenue', 'grossmargin', 'netinc', 'bvps', 'fcf', 'fcfps', 'equity', 'eps', 'sps', 'ros', 'roe', 'roic', 'payoutratio', 'ebitdamargin']

    fundamental_data = nasdaqdatalink.get_table('SHARADAR/SF1', dimension="MRT", qopts={"columns":full_figure_list}, ticker=ticker)  
    df = pd.DataFrame(fundamental_data)

    
    if not df.empty:
        fund_df = df.iloc[::-1]

        # Grab most recent observation
        first_row = fund_df.iloc[-1]
        print(first_row)
        for metric in latest_observation_list:
            value = first_row[metric]
            if value and value > 0:
                metric_perc = (value - industry_average_row[metric]) / industry_average_row[metric]
                if metric_perc:
                    row_values[metric] = metric_perc

        # Here we'll calculate percentage difference between 
        for growth_metric in growth_rates_list:
            column = fund_df[growth_metric]
            filtered_column = column.dropna()
            # filter out zero or near-zero values
            filtered_column = filtered_column[filtered_column > 0.001]

            # We want to calculate only companies that have a significant history so our
            # average growth rates are based off at least 5 years. We are using TTM so we
            # are calculated the TTM each quarter 4 * 5 = 20 entries
            if len(filtered_column) > 19:
                simple_rates = (filtered_column - filtered_column.shift(1)) / filtered_column.shift(1)
                average_growth_rate = np.nanmean(simple_rates)
                if not np.isnan(average_growth_rate):
                    growth_rate_perc = (average_growth_rate - industry_average_row[f'{growth_metric}_average_growth_rate']) / industry_average_row[f'{growth_metric}_average_growth_rate']
                    row_values[f'{growth_metric}_average_growth_rate'] = growth_rate_perc

    percentage_dist.append(row_values)
    # time.sleep(0.05)

percentage_dist_df = pd.DataFrame(percentage_dist)

# Write DataFrame to CSV file
csv_file_path = 'perc_distance.csv'
percentage_dist_df.to_csv(csv_file_path, index=False)



revenue                NaN
netinc                 NaN
bvps                  2.51
rnd                    NaN
fcf                    NaN
equity           9980831.0
assetturnover          NaN
currentratio         0.091
de                   0.314
divyield              None
eps                    NaN
fcfps                  NaN
grossmargin           None
ebitdamargin          None
sps                    NaN
ros                   None
roic                   NaN
roe                    NaN
roa                    NaN
payoutratio            NaN
Name: 0, dtype: object
revenue            NaN
netinc             NaN
bvps             0.008
rnd                NaN
fcf                NaN
equity           21301
assetturnover     None
currentratio     0.046
de               6.449
divyield          None
eps                NaN
fcfps              NaN
grossmargin       None
ebitdamargin      None
sps                NaN
ros               None
roic              None
roe               None
roa               None


KeyError: nan